# STRATIS Workflow Demo from PIXNAT data

In [9]:
import os, oct2py, octave_kernel, requests, shutil, time
from getpass import getpass
from pyxnat import Interface

%load_ext oct2py.ipython

The oct2py.ipython extension is already loaded. To reload it, use:
  %reload_ext oct2py.ipython


In [10]:
xusr = 'locastre'
xpwd = getpass()

 ········


In [11]:
xhost = 'http://plvimphxnat1'
proj = '22547_STRATIS_Demo_1'

In [12]:
xnat = Interface(xhost,xusr,xpwd)
xproj = xnat.select.project(proj)
xsubjlist = xproj.subjects('*')

In [ ]:
cerrPath = '/cluster/home/locastre/CERR_master/CERR'

In [ ]:
%octave_push cerrPath

In [ ]:
workingdir = '/cluster/home/locastre/STRATIS_demo_1'
os.makedirs(workingdir,exist_ok=True)

## Download files from each subject in PIXNAT experiment

In [ ]:
session_list = []

for xsubj in xsubjlist:
    subj = xsubj.attrs.get('label')
    xexplist = xsubj.experiments('*')
    subjdir = os.path.join(workingdir,subj)
    os.makedirs(subjdir,exist_ok=True)
    for xexp in xexplist:
        sessionlabel = xexp.attrs.get('label')
        xscanlist = xexp.scans('*')
        savedir = os.path.join(subjdir,sessionlabel)
        os.makedirs(savedir,exist_ok=True)
        cerrdir = os.path.join(savedir,'CERR')
        os.makedirs(cerrdir,exist_ok=True)
        session_list.append(savedir)
        os.makedirs(savedir,exist_ok=True)
        for xscan in xscanlist:
            try:
                xscan.resource('DICOM').get(savedir,extract=True)
            except:
                xscan.resource('secondary').get(savedir,extract=True)
                secfile = os.listdir(os.path.join(savedir,'secondary'))
                for sf in secfile:
                    shutil.move(os.path.join(savedir,'secondary',sf),os.path.join(savedir,'DICOM',sf))

In [ ]:
%octave_push savedir dicomdir session_list

In [ ]:
%%octave

addpath(cerrPath);
addToPath2(cerrPath);

init_ML_DICOM;

for i = 1:numel(session_list)
    
planC = importDICOM(dicomdir);
mkdir(fullfile(savedir,'CERR'));
planCfile = fullfile(savedir,'CERR','planC.mat');
save_planC(planC,[],'PASSED',planCfile);
end

In [ ]:
%octave_pull planCfile

## Create resource folders and upload results to PIXNAT via REST and pyxnat(template)

Authenticate to PIXNAT via requests module with user credentials

In [ ]:
session = requests.Session()
session.auth = (xusr, xpwd)
auth = session.post(xhost)

The name of the folder is "CERR" here

In [ ]:
xres = xexp.resource('CERR')

In [ ]:
if not xres.exists():
    req_URL = xhost + '/data/experiments/' + exp + '/resources/CERR'
    print(req_URL)
    r = session.put(req_URL)
    r.content

In [ ]:
xfile = xres.file('planC.mat')
if not xfile.exists():
    xfile.insert(planCfile,'planC.mat')

Create new scan object and upload secondary type DICOM images

In [ ]:
newxscan = xexp.scan('CERR-1')
newxscan.insert()
newxscan.resource('secondary').create()
newxscan.resource('secondary').put_dir(cerrdcmpath)

Add RTSTRUCT to the ROI collection in PIXNAT

In [ ]:
def rt_mod(rtstructfile):
    rtdcm = pydicom.dcmread(rtstruct)
    if rtdcm.StudyDate=='':
        rtdcm.StudyDate=d.StructureSetDate

    if rtdcm.StructureSetTime=='':
        rtdcm.StructureSetTime=time.strftime('%H%M%S') + '.00'

    if rtdcm.StudyTime=='':
        rtdcm.StudyTime=d.StructureSetTime

    try:
        agestr = d['PatientAge'].value
        if agestr != '':
            patientAge = int(agestr[:-1])
            if patientAge > 100:
                rtdcm['PatientAge'].value = '099Y'
    except:
        print('No PatientAge tag')
    rtdcm.save_as(rtstructfile)

In [ ]:
rtstructfile = ''

rtdcm = pydicom.dcmread(rtstructfile)
xuid = rtdcm['Series Description'].value.replace(' ','_')


headers = {'Content-Type':'application/octet-stream','Accept':'text/plain'}

req_URL = xhost + '/xapi/roi/projects/' + xproj + '/sessions/' + exp + '/collections/' + xuid + '?type=RTSTRUCT&overwrite=true"

r = session.put(req_URL,headers=headers,file=rtstructfile)
r.contents

dfsdfsdfsdfsd